In [3]:
#imports
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


SyntaxError: invalid syntax (3120868670.py, line 2)

In [ ]:
#claves

In [11]:
#codigo 
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.send"]


def main():
  """Shows basic usage of the Gmail API.
  Lists the user's Gmail labels.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("No labels found.")
      return
    print("Labels:")
    for label in labels:
      print(label["name"])

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")




if __name__ == "__main__":
  main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=176816052410-qqmi7d448pic80ns0b51m06gq2u0m073.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49358%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=qlD9pILEGV0wDaeQdWU47jZK0W2aOy&access_type=offline
An error occurred: <HttpError 403 when requesting https://gmail.googleapis.com/gmail/v1/users/me/labels?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">


In [13]:
import os.path
import base64
import datetime
from email.mime.text import MIMEText

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


SCOPES = ["https://www.googleapis.com/auth/gmail.send"]

def get_credentials():
    """Autentica con OAuth2 y devuelve las credenciales."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return creds

def send_email():
    """Envía un correo usando la API de Gmail."""
    try:
        # Autenticación
        creds = get_credentials()
        service = build("gmail", "v1", credentials=creds)

        # Configurar destinatario y mensaje
        sender_email = "cadenaseficientesteam@gmail.com"  # Cambia esto por tu correo
        receiver_email = "luis.l.g.mejia327@gmail.com"  # Cambia esto por el destinatario
        subject = "Prueba 1 de Gmail"
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        body = f"Este es un mensaje de prueba usando Gmail {current_time}"

        # Crear el mensaje MIME
        message = MIMEText(body)
        message["to"] = receiver_email
        message["from"] = sender_email
        message["subject"] = subject

        # Codificar el mensaje en base64
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

        # Enviar el correo
        message = {"raw": raw_message}
        sent_message = service.users().messages().send(userId="me", body=message).execute()

        print(f"📩 Correo enviado con ID: {sent_message['id']}")

    except HttpError as error:
        print(f"❌ Error al enviar el correo: {error}")

if __name__ == "__main__":
    send_email()



📩 Correo enviado con ID: 1959d35e1101e901
